# Продвинутое машинное обучение
## Домашнее задание 2

In [1]:
import pickle as pkl
from datetime import datetime
from collections import defaultdict
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression, LinearRegression
from scipy.stats import kendalltau, spearmanr


from tqdm import tqdm

In [2]:
tournaments = pkl.load(open('tournaments.pkl', 'rb'))
results = pkl.load(open('results.pkl', 'rb'))
players = pkl.load(open('players.pkl', 'rb'))

**1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:**
 
*   **взять в тренировочный набор турниры с dateStart из 2019 года;**
*   **в тестовый — турниры с dateStart из 2020 года.**

In [4]:
train = {
    'player_id': [],
    'player_name': [],
    'player_rating': [],
    
    'team_id': [],
    'team_position': [],
    
    'tournament_id': [],
    'tournament_name': [],
    'question_order': [],
    'is_answered': []
}
test = {
    'player_id': [],
    'player_name': [],
    'player_rating': [],
    
    'team_id': [],
    'team_position': [],
    
    'tournament_id': [],
    'tournament_name': [],
    'question_order': [],
    'is_answered': []
}

In [5]:
for key, tournament in tqdm(tournaments.items(), position=0, leave=True):
    not_defined_idxs = []
    if datetime.fromisoformat(tournament['dateStart']).year not in [2019, 2020]:
        continue
    # Определим номера вопросов, где хотя бы у одной команды была метка '?' или 'X', чтобы исключить потом
    for team_result in results[key]:
        if 'mask' in team_result.keys():
            if team_result['mask']:
                for i, is_answered in enumerate(team_result['mask']):
                    if is_answered in ['X', '?']:
                        not_defined_idxs.append(i)
    not_defined_idxs = np.unique(np.array(not_defined_idxs))

    for team_result in results[key]:
        for player in team_result['teamMembers']:
            if 'mask' in team_result.keys():
                if team_result['mask']:
                    for i, is_answered in enumerate(team_result['mask']):
                        if i not in not_defined_idxs:
                            if datetime.fromisoformat(tournament['dateStart']).year == 2019:
                                train['player_id'].append(player['player']['id'])
                                train['player_name'].append(f"{player['player']['surname']} {player['player']['name']} {player['player']['patronymic']}")
                                train['player_rating'].append(player['rating'])

                                train['team_id'].append(team_result['team']['id'])
                                train['team_position'].append(team_result['position'])

                                train['tournament_id'].append(key)
                                train['tournament_name'].append(tournament['name'])

                                train['question_order'].append(i)
                                train['is_answered'].append(int(team_result['mask'][i]))
                                
                            elif datetime.fromisoformat(tournament['dateStart']).year == 2020:
                                test['player_id'].append(player['player']['id'])
                                test['player_name'].append(f"{player['player']['surname']} {player['player']['name']} {player['player']['patronymic']}")
                                test['player_rating'].append(player['rating'])

                                test['team_id'].append(team_result['team']['id'])
                                test['team_position'].append(team_result['position'])

                                test['tournament_id'].append(key)
                                test['tournament_name'].append(tournament['name'])

                                test['question_order'].append(i)
                                test['is_answered'].append(int(team_result['mask'][i]))

100%|██████████████████████████████████████████████████████████████████████████████| 5528/5528 [02:30<00:00, 36.71it/s]


In [6]:
train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

In [7]:
# train_df.to_csv('train.csv', index=False)
# test_df.to_csv('test.csv', index=False)

In [46]:
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

**2. Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:**
* **повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;**
* **в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;**
* **для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.**

Обучим модель логистической регрессии на следующих атрибутах:
* доля взятых игроком вопросов за все время от общего числа вопросов по всем отыгранным турнирам и количество отыгранных вопросов - для неявного учета в модели силы игрока;
* доля игроков, взявших вопрос в текущем турнире об общего числа вопросов в турнире - для неявного учета сложности вопроса в рамках турнира.

In [47]:
train_par = train_df[['player_id', 'is_answered']]\
                    .groupby('player_id')\
                    .agg('mean').rename(columns={'is_answered': 'player_answer_rate'})
train_tar = train_df[['tournament_id', 'is_answered']]\
                    .groupby('tournament_id')\
                    .agg('mean').rename(columns={'is_answered': 'tournament_answer_rate'})

train_df = train_df.merge(train_par, how='left', left_on='player_id', right_index=True)\
                   .merge(train_tar, how='left', left_on='tournament_id', right_index=True)\

train_df['global_answer_rate'] = train_df['is_answered'].mean()

In [48]:
X_train = train_df[['player_answer_rate', 'global_answer_rate', 'tournament_answer_rate']]
y_train = train_df['is_answered']

In [49]:
%%time
model = LogisticRegression(n_jobs=-1)
model.fit(X_train, y_train)

Wall time: 40.3 s


LogisticRegression(n_jobs=-1)

In [50]:
train_df['predict'] = model.predict_proba(train_df[['player_answer_rate', 'global_answer_rate', 
                                                    'tournament_answer_rate']])[:, 1]

**3. Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:**
* **предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;**
* **в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.**

Для предсказания результатов турнира с неизвестными вопросами предлагается создание искусственных турниров из 36 вопросов с частотой взятий от "гроба", до "свечки", увеличивающейся с фиксированным шагом.
Рейтинг команды будет определяться как сумма вероятностей ответа команды на каждый вопрос в турнире.
Вероятность ответа на один вопрос: $1 - \Pi_{i \in k}(1 - pr_{i})$
, где $pr_{i}$ - предсказанная моделью вероятность игрока i из команды k ответить на текущий вопрос

In [51]:
def estimate_quality():
    test_df_train = test_df[test_df['player_id'].isin(train_df['player_id'].unique())]
    test_teams = test_df_train[['player_id', 'team_id', 'tournament_id', 'team_position']].drop_duplicates()

    generated_test = {
        'player_id': [], 
        'team_id': [], 
        'tournament_id': [], 
        'team_position': [],
        'question_order': [],
        'tournament_answer_rate': []
    }
    for row in test_teams.iterrows():
        for i in range(36):
            generated_test['player_id'].append(row[1]['player_id'])
            generated_test['team_id'].append(row[1]['team_id'])
            generated_test['tournament_id'].append(row[1]['tournament_id'])
            generated_test['team_position'].append(row[1]['team_position'])
            generated_test['question_order'].append(i)
            generated_test['tournament_answer_rate'].append(i * 1/35)


    generated_test_df = pd.DataFrame(generated_test)\
                    .merge(train_df[['player_id', 'player_answer_rate', 'global_answer_rate']].drop_duplicates(),
                           how='left', on='player_id')
    if str(type(model)).find('LinearRegression') != -1:
        generated_test_df['predict'] = predict(model, generated_test_df[['player_answer_rate', 'global_answer_rate', 
                                                            'tournament_answer_rate']])
    else:
        generated_test_df['predict'] = model.predict_proba(generated_test_df[['player_answer_rate', 'global_answer_rate', 
                                                            'tournament_answer_rate']])[:, 1]
    generated_test_df['predict_inv'] = 1 - generated_test_df['predict']

    ranks = generated_test_df[['team_id', 'tournament_id', 'question_order', 'predict_inv', 'team_position']]\
                     .groupby(['team_id', 'tournament_id', 'question_order', 'team_position']).agg('prod')\
                     .groupby(['team_id', 'tournament_id', 'team_position']).agg('sum').reset_index()

    ranks['rank'] = ranks.groupby('tournament_id')['predict_inv'].rank("dense", ascending=True)

    sp_corrs, kend_corrs = [], []
    for tournament_id in ranks['tournament_id'].unique():
        tournament_res = ranks[ranks['tournament_id'] == tournament_id]
        kend_corr = kendalltau(tournament_res['rank'], tournament_res['team_position'])[0]
        sp_corr = spearmanr(tournament_res['rank'], tournament_res['team_position'])[0]
        if sp_corr != 'nan' and kend_corr != np.nan:
            kend_corrs.append(kend_corr)
            sp_corrs.append(sp_corr)

    sp_corrs, kend_corrs = np.array(sp_corrs), np.array(kend_corrs)

    sp_corrs = sp_corrs[~np.isnan(sp_corrs)]
    kend_corrs = kend_corrs[~np.isnan(kend_corrs)]

    print(f"Корреляция Спирмена: {np.array(sp_corrs).mean()}")
    print(f"Корреляция Кендалла: {np.array(kend_corrs).mean()}")

In [52]:
estimate_quality()

Корреляция Спирмена: 0.6971294578149314
Корреляция Кендалла: 0.5413885756876532


**4. Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:**
* **предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;**
* **разработайте EM-схему для обучения этой модели, реализуйте её в коде;**
* **обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.**

Пусть $z_{ij}$ - скрытая переменная, ответил ли i-ый игрок на j-ый вопрос, $y_{kj}$ - видимая переменная, ответила ли k-ая команда на j-ый вопрос.

Тогда на E-шаге предлагается проводить следующее обновление:
1. Если $y_{kj} = 0$, то $E[z_{ij}] = 0$ для всех $i \in k$
2. Если $y_{kj} \neq 0$, то $E[z_{ij}]$ = p(ответил игрок|команда ответила) = p(команда ответила|ответил игрок) * p(ответил игрок) / p(команда ответила) =
$$=\frac{1 * pr_i}{1 - \Pi_{i \in k}(1 - pr_{i})}$$
, где $pr_{i}$ - предсказанная моделью вероятность игрока i из команды k ответить на текущий вопрос

In [53]:
def fit_model(lr, X_train, y_train):
    y = y_train.copy(deep=True)

    y = np.clip(y, 1e-8, 1 - 1e-8)
    inv_sig_y = np.log(y / (1 - y))

    lr.fit(X_train, inv_sig_y)
    return lr

def sigmoid(x):
    ex = np.exp(x)
    return ex / (1 + ex)

def predict(lr, X_test):
    return sigmoid(lr.predict(X_test))

In [54]:
X_train = train_df[['player_answer_rate', 'global_answer_rate', 'tournament_answer_rate']]
y_train = train_df['is_answered']

In [55]:
def m_step():
    lr = LinearRegression(n_jobs=-1)
    lr = fit_model(lr, X_train, y_train)
    train_df['predicted_proba'] = predict(lr, X_train)
    return lr

In [56]:
def e_step():
    train_df['inv_predicted_proba'] = 1 - train_df['predicted_proba']
    new_target = train_df[['tournament_id', 'team_id', 'predicted_proba', 'question_order']].groupby(['tournament_id', 'team_id', 'question_order'])\
                            .agg('prod').reset_index().rename(columns={'predicted_proba': 'new_target'})
    new_target['new_target'] = 1 - new_target['new_target']
    new_target = new_target.merge(train_df[['tournament_id', 'team_id', 'is_answered', 'question_order']], how='left', 
                                  on=['tournament_id', 'team_id', 'question_order'])
    new_target.loc[new_target['is_answered'] == 0, 'new_target'] = 0
    return new_target['new_target']

In [57]:
for i in range(5):
    print(f"Шаг {i+1}")
    model = m_step()
    y_train = e_step()
    estimate_quality()
    print()

Шаг 1
Корреляция Спирмена: 0.7054783328994335
Корреляция Кендалла: 0.5506333528396041

Шаг 2
Корреляция Спирмена: 0.6971201209711633
Корреляция Кендалла: 0.5418878668652158

Шаг 3
Корреляция Спирмена: 0.7301673595509643
Корреляция Кендалла: 0.5725300561078268

Шаг 4
Корреляция Спирмена: 0.7322513976011555
Корреляция Кендалла: 0.5746277763399601

Шаг 5
Корреляция Спирмена: 0.7301624603421936
Корреляция Кендалла: 0.5724944459858189



## Результат

По сравнению с результатами baseline'a (см. п. 3):

* Корреляция Спирмена: 0.6971294578149314
* Корреляция Кендалла: 0.5413885756876532

качество модели, обученной с помощью EM-схемы, увеличилось.

**5. А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)?**

Для формирования рейтинга турнира расчитаем на основе полученных с помощью моделей вероятностей *игрока* ответить на вопрос вероятности *команды* ответить на вопрос.

Итоговый рейтинг турнира будет рассчитан как сумма вероятностей всех команд ответить на каждый вопрос турнира.

In [62]:
train_df['predicted_proba'] = predict(model, X_train)
train_df['predict_inv'] = 1 - train_df['predict']

In [77]:
tournaments_rating = train_df[['team_id', 'tournament_id', 'question_order', 'predict_inv', 'tournament_name']]\
                 .groupby(['team_id', 'tournament_id', 'question_order', 'tournament_name']).agg('prod').reset_index()
tournaments_rating = tournaments_rating[['tournament_id', 'predict_inv', 'tournament_name']]\
                 .groupby(['tournament_id', 'tournament_name']).agg('sum')
tournaments_rating.sort_values('predict_inv')

,,predict_inv
tournament_id,tournament_name,
5945,Чемпионат Мира. Этап 3. Группа А,0.081328
5948,Чемпионат Мира. Финал. Группа А,0.086169
5941,Чемпионат Мира. Этап 2. Группа А,0.230880
5827,Шестой киевский марафон. Асинхрон,0.335125
5963,Асинхрон по South Park,0.708476
...,...,...
5852,Лига вузов. I тур,10483.491939
5025,Кубок городов,10539.139078
6255,ОВСЧ,11069.079088


Построенный рейтинг частично соответствует логике: в топ-5 с наивысшим рейтингом попали 3 чемпионата мира, в топ-5 с наименьшим рейтингом попали школьные и вузовские лиги.